In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\stali\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

In [3]:
def answer_conc(df):

  df_questions = df[df['ParentId'].isna()]
  df_answers = df[df['ParentId'].notna()]

  df_answers = df_answers.drop(columns=['AcceptedAnswerId','ViewCount','Tags'])

  df_questions = df_questions.merge(df_answers[['Id','Text']], how='left', left_on='AcceptedAnswerId', right_on='Id', suffixes=('', '_Answer'))
  df_questions.rename(columns={'Text_Answer': 'Answer'}, inplace=True)
  df_questions = df_questions.drop(columns='Id_Answer')
  df_answers_reduced = df_answers.loc[df_answers.groupby('ParentId')['Score'].idxmax()]
  df_no_accepted_answer = df_questions[df_questions['AcceptedAnswerId'].isna()]

  for index, row in df_no_accepted_answer.iterrows():
    question_id = row['Id']
    highest_scored_answer = df_answers_reduced[df_answers_reduced['ParentId'] == question_id]
    if not highest_scored_answer.empty:
        df_questions.at[index, 'Answer'] = highest_scored_answer.iloc[0]['Text']
  return df_questions

def process_tags(tags):
    if pd.isna(tags):
      return []
    tags_list = tags.split('|')
    return tags_list[1:-1] if len(tags_list) > 2 else []

def big_clean(df):
  df = df.drop(columns=['LastActivityDate','CreationDate','PostTypeId','OwnerUserId','ContentLicense','CommentCount','AnswerCount'])
  df['Text'] = df['Title'].fillna('') + ' ' + df['Body'].fillna('').str.strip()
  df = df.drop(columns=['Body','Title'])

  questions = answer_conc(df)

  questions.rename(columns={'Text':'Question'},inplace=True)
  questions = questions.drop(columns=['ViewCount','AcceptedAnswerId','ParentId','Score'])

  questions['Tags'] = questions['Tags'].apply(process_tags)

  return questions

In [5]:
df_chess = pd.read_csv('https://github.com/yarathealmighty/dumps/blob/main/data/parsed/chess.csv?raw=true')
df_beer = pd.read_csv('https://github.com/yarathealmighty/dumps/blob/main/data/parsed/beer.csv?raw=true')
df_coffee = pd.read_csv('https://github.com/yarathealmighty/dumps/blob/main/data/parsed/coffee.csv?raw=true')
df_movies = pd.read_csv('https://github.com/yarathealmighty/dumps/blob/main/data/parsed/movies.csv?raw=true')

In [6]:
chess = big_clean(df_chess)
beer = big_clean(df_beer)
coffee = big_clean(df_coffee)
movies = big_clean(df_movies)

KeyError: 'ParentId'